In [52]:
from google.colab import drive
drive.mount('/content/driver')

Drive already mounted at /content/driver; to attempt to forcibly remount, call drive.mount("/content/driver", force_remount=True).


In [53]:
import pandas as pd

In [54]:
df_data = df_data = pd.read_excel('/content/driver/MyDrive/MAT308/data (2).xlsx')
del df_data['Unnamed: 2']

In [55]:
df_data

,Comment,Labels
0,máy bụi nhiều và nhìn hơi cũ. giá 170k thấy hơ...,0
1,Máy mới xài tạm ổn. Máy này xây hạt thành bột ...,0
2,"Giao hàng nhanh, đúng lịch. Nhưng k có túi nil...",0
3,"hài lòng hàng như quảng cáo cáo , nhưng có điề...",0
4,"Nắp đậy không kín, xay bị văn bột ra ngoài",0
...,...,...
1639,Được giới thiệu từ quyển Kinh doanh online trê...,1
1640,tôi đã từng đọc quyển sách khởi nghiệp bán lẻ ...,1
1641,"Quả thật, sản phẩm nào cũng anh Phong cũng chấ...",1
1642,"Thực sự rất bất ngờ với tốc độ giao của sách, ...",1


In [ ]:
df = pd.read_csv('/content/driver/MyDrive/MAT308/data - data.csv (2).csv', delimiter='\t', header=None)

In [ ]:
df

,0
0,máy bụi nhiều và nhìn hơi cũ. giá 170k thấy hơ...
1,Máy mới xài tạm ổn. Máy này xây hạt thành bột ...
2,"Giao hàng nhanh, đúng lịch. Nhưng k có túi nil..."
3,"hài lòng hàng như quảng cáo cáo , nhưng có điề..."
4,"Nắp đậy không kín, xay bị văn bột ra ngoài,0,"
...,...
1639,Được giới thiệu từ quyển Kinh doanh online trê...
1640,tôi đã từng đọc quyển sách khởi nghiệp bán lẻ ...
1641,"Quả thật, sản phẩm nào cũng anh Phong cũng chấ..."
1642,"Thực sự rất bất ngờ với tốc độ giao của sách, ..."


In [56]:
import re

def standardize_data(row):
    # remove stopword
    # Remove . ? , at index final
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Remove all . , " ... in sentences
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")

    row = row.lower()
    row = row.strip()

    return row

In [57]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
from transformers import BertModel, BertTokenizer
import joblib

In [59]:
# load model BERT
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#encode lines
tokenized = df_data['Comment'].apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))
print('encode',tokenized[1])
# decode
print('decode',tokenizer.decode(tokenized[1]))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


encode [101, 2089, 25175, 1060, 4886, 17214, 2006, 1012, 2089, 29349, 1060, 4710, 6045, 2084, 2232, 28516, 18699, 2319, 2232, 1010, 18699, 5575, 11265, 2226, 14163, 2239, 1060, 4710, 18699, 26230, 2368, 10189, 2084, 2232, 8945, 11320, 2239, 16215, 2072, 1047, 19991, 1102, 19098, 2278, 1010, 2079, 2053, 12170, 17710, 2102, 12436, 2080, 11320, 10448, 4830, 2080, 1010, 12731, 3070, 2089, 10023, 26230, 3388, 1013, 1011, 2154, 1102, 9013, 11586, 2232, 12436, 2080, 2089, 11265, 2078, 1047, 6806, 2050, 2310, 8254, 2232, 102]
decode [CLS] may moi xai tam on. may nay xay hat thanh bot nhanh, nhung neu muon xay nhuyen hon thanh bo luon thi khong đuoc, do no bi ket vao luoi dao, cung maykhuyet / - day đien dinh vao may nen khoa ve sinh [SEP]


In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
#get all label 
labels = np.zeros(len(df_data['Labels']))
for i in range(1,len(df_data['Labels'])):
    labels[i] = df_data['Labels'][i]
print('labels shape:', labels.shape)

# get lenght max of tokenized
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print('max len:', max_len)

# if lenght of tokenized not equal max_len , so padding value 0
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print('padded:', padded[1])
print('len padded:', padded.shape)

#get attention mask ( 0: not has word, 1: has word)
attention_mask = np.where(padded ==0, 0,1)
print('attention mask:', attention_mask[1])

# Convert input to tensor
padded = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)


# Train model
batch = 26
batch_size = 64
list = []
for i in range(1,batch+1):
  index_beffor = (i-1) * batch_size 
  if i == 26:
    index_affter == 1644
  index_affter = i * batch_size
  input_ids_batch = padded[index_beffor:index_affter]
  attention_mask_batch = attention_mask[index_beffor:index_affter]
  with torch.no_grad():
      last_hidden_states = model(input_ids = input_ids_batch, attention_mask = attention_mask_batch)
  #     print('last hidden states:', last_hidden_states)

  features = last_hidden_states[0][:,0,:].numpy()
  list.append(features)
  print("batch ", i)
  print('features:', features)

# X_train, X_test, y_train, y_test = train_test_split(features, labels)

# cl = LogisticRegression()
# cl.fit(X_train, y_train)

# # Save model
# joblib.dump(cl, 'save_model.pkl')
# sc = cl.score(X_test, y_test)
# print('score:', sc)

labels shape: (1644,)
max len: 258
padded: [  101  2089 25175  1060  4886 17214  2006  1012  2089 29349  1060  4710
  6045  2084  2232 28516 18699  2319  2232  1010 18699  5575 11265  2226
 14163  2239  1060  4710 18699 26230  2368 10189  2084  2232  8945 11320
  2239 16215  2072  1047 19991  1102 19098  2278  1010  2079  2053 12170
 17710  2102 12436  2080 11320 10448  4830  2080  1010 12731  3070  2089
 10023 26230  3388  1013  1011  2154  1102  9013 11586  2232 12436  2080
  2089 11265  2078  1047  6806  2050  2310  8254  2232   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0  

In [62]:
data_train = list[0]
for i in range(1, len(list)):
  data_train = np.concatenate((data_train, list[i]), axis=0)


In [63]:
data_train = np.reshape(data_train,(1644, 768))

In [64]:
data_train.shape

(1644, 768)

In [68]:
import pickle

In [70]:
X_train, X_test, y_train, y_test = train_test_split(data_train, labels)

cl = LogisticRegression()
cl.fit(X_train, y_train)

# Save model
pickle.dump(cl, open('save_model.pkl', 'wb'))
sc = cl.score(X_test, y_test)
print('score:', sc)

score: 0.8199513381995134


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
y_train.isn


(1233,)

In [51]:
X_train.shape

(1233, 768)